In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from statsmodels.datasets import sunspots

def load_dataset(name: str, test_size: float = 0.2, normalize: bool = True):
    """
    Load and preprocess a time series dataset.

    Parameters:
    - name: Name of the dataset ("airpassengers", "sunspots", "electricity", "covid")
    - test_size: Fraction of data to be used for testing
    - normalize: Whether to normalize the data using MinMaxScaler

    Returns:
    - train_series: Training part of the series
    - test_series: Testing part of the series
    """
    name = name.lower()

    if name == "airpassengers":
        # Simulate monthly passenger numbers with seasonality and trend
        np.random.seed(0)
        time = np.arange(144)
        series = 100 + 10 * np.sin(2 * np.pi * time / 12) + 0.5 * time + np.random.normal(scale=5, size=144)

    elif name == "sunspots":
        df = sunspots.load_pandas().data
        series = df['SUNACTIVITY'].values

    elif name == "electricity":
        # Simulated electricity consumption with daily/weekly seasonality
        np.random.seed(1)
        time = np.arange(1000)
        series = 200 + 20 * np.sin(2 * np.pi * time / 24) + 5 * np.sin(2 * np.pi * time / (24*7)) + np.random.normal(scale=10, size=1000)

    elif name == "covid":
        # Simulated COVID-19 daily case counts with waves
        np.random.seed(2)
        time = np.arange(500)
        series = (
            1000 * np.exp(-((time - 100) ** 2) / 2000) +
            2000 * np.exp(-((time - 300) ** 2) / 3000) +
            np.random.normal(scale=50, size=500)
        )

    else:
        raise ValueError(f"Unknown dataset name: {name}")

    series = series.astype(float).reshape(-1, 1)
    if normalize:
        scaler = MinMaxScaler()
        series = scaler.fit_transform(series).flatten()

    train, test = train_test_split(series, test_size=test_size, shuffle=False)

    return train, test

# Example test
#load_dataset("sunspots")


In [ ]:
def train_and_predict(model_name: str, train_series, test_steps: int):
    """
    Fit the model and return predictions.
    """
    if model_name == "ARIMA":
        # Fit ARIMA, return forecast
    elif model_name == "LSTM":
        # Prepare LSTM, return forecast
    # ... etc.
    return y_pred, y_true


In [ ]:
def evaluate_forecast(y_true, y_pred):
    """
    Compute and return all evaluation metrics.
    """
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    smape = 100 * np.mean(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)))
    
    return {
        "MAE": mae,
        "RMSE": rmse,
        "MAPE": mape,
        "sMAPE": smape
    }


In [ ]:
def benchmark_all(models, datasets):
    """
    Run all models on all datasets and store results.
    """
    results = []
    for dataset in datasets:
        train, test = load_dataset(dataset)
        for model in models:
            try:
                y_pred, y_true = train_and_predict(model, train, len(test))
                metrics = evaluate_forecast(y_true, y_pred)
                results.append({
                    "Dataset": dataset,
                    "Model": model,
                    **metrics
                })
            except Exception as e:
                print(f"Error with {model} on {dataset}: {e}")
    return pd.DataFrame(results)
